# **Sesión 1:** Calibración de Cámara 📷⚙️

## **Instalaciones**

In [60]:
!pip install imageio opencv-python


## **Librerías**

In [61]:
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob
import os


In [62]:
import sys
print(sys.executable)

C:\Users\Claudia\Documents\3º IMAT\Visión por ordenador I\Práctica 1\Lab_1\practicas_vision\Scripts\python.exe


## **Apartado A: Calibración de cámara** (derecha e izquierda)

En este apartado se realiza la calibración de dos cámaras de un sistema estereoscópico. Para ello se trabajará con las imágenes de las carpetas ``left`` y ``right``. En primer lugar se trabajará con la carpeta ``left``. Posteriormente, deberá repetir el proceso con las imágenes en la carpeta ``right``. Ambas carpetas contienen imágenes con las que se calibrarán las cámaras. En ellas aparece el patrón de calibración en diferentes posiciones y orientaciones. Estas imágenes serán los datos de entrada.

Los pasos que deberá seguir para calibrar una cámara son:

1. Defina y ejecute el método para cargar imágenes ``load_images()``.
2. Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.
3. Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``.
4. Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)`` que proporcione las coordenadas 3D de las esquinas del patrón. El sistema de referencia utilizado deberá estar anclado al propio patrón.
5. Utilice ``cv2.calibrateCamera`` para obtener los parámetros de calibración para la cámara izquierda.

### **Tarea A.1:** Defina y ejecute el método para cargar imágenes ``load_images()``.

In [63]:
def load_images(filenames: List) -> List:
    return [imageio.v2.imread(filename) for filename in filenames]

In [64]:
# TODO Build a list containing the paths of all images from the left camera
imgs_path = []
for i in range(19):
    if i <10:
        imgs_path.append(f'../data/left/left_00{i}.jpg')
    else:
        imgs_path.append(f'../data/left/left_0{i}.jpg')
        
        

imgs = load_images(imgs_path)


### **Tarea A.2:** Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.

In [65]:
# TODO Find corners with cv2.findChessboardCorners()
corners_list = []
for i in range(19):
    corners = cv2.findChessboardCorners(imgs[i],(8,6))
    corners_list.append(corners)


In [66]:
corners_copy = copy.deepcopy(corners_list)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.


def to_gray(imgs):
    list_imgs_gray = [] 
    for img in imgs:
        imgs_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        list_imgs_gray.append(imgs_gray)
    return list_imgs_gray

list_imgs_gray = to_gray(imgs)

def refine_corners(list_imgs_gray,corners_copy,criteria):
    corners_refined = [cv2.cornerSubPix(i, cor[1], (8, 6), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(list_imgs_gray, corners_copy)]
    return corners_refined


corners_refined = refine_corners(list_imgs_gray,corners_copy,criteria)

### **Tarea A.3:** Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``

In [67]:
imgs_copy = copy.deepcopy(imgs)

In [68]:
# TODO Use cv2.drawChessboardCorners() to draw the cornes
corners_refined_copy = corners_refined.copy()
imgs_copy = imgs.copy()

def draw_corners(images: List[np.ndarray], corners: List[np.ndarray], pattern_size) -> List[np.ndarray]:
    images_with_corners = []
    for img, corner in zip(images, corners):
        if corner.size > 0:
            img_with_corners = cv2.drawChessboardCorners(img, pattern_size, corner, True)
            images_with_corners.append(img_with_corners)
            
    return images_with_corners

images = draw_corners(imgs_copy,corners_refined_copy,(8,6))

In [69]:
# TODO Show images and save when needed

def show_image(window_name,image):
    cv2.imshow(window_name,image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def write_image(filename:str,image):
    cv2.imwrite(filename,image)

In [70]:
for i in range(19):
        if i <10:
            show_image(f"Left_00{i}",images[i])
            write_image(f'../src/left_00{i}.jpg',images[i])
        else:
            show_image(f"Left_0{i}",images[i])
            write_image(f'../src/left_0{i}.jpg',images[i])

### **Tarea A.4:** Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)``

In [71]:
# TODO Design the method. It should return a np.array with np.float32 elements
def get_chessboard_points(chessboard_shape, dx, dy):
    coor = []
    n_cols = chessboard_shape[0] 
    n_rows = chessboard_shape[1]  

    for i in range(n_rows):
        for j in range(n_cols):
            coor.append([j * dx, i * dy, 0])  
    return np.array(coor, dtype=np.float32)


In [99]:
# TODO You need the points for every image, not just one
chessboard_points = get_chessboard_points((8, 6), 30, 30)

valid_chessboard_points = [get_chessboard_points((8, 6), 30, 30) for i in range(len(images))]

### **Tarea A.5:** Utilice ``cv2.calibrateCamera()`` para obtener los parámetros de calibración para la cámara izquierda

In [73]:
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners_list if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [74]:
# Chessboard points for every image
valid_chessboard_points = [chessboard_points for _ in range(len(valid_corners))]

In [75]:
# TODO
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(valid_chessboard_points, valid_corners, list_imgs_gray[0].shape[::-1], None, None)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

In [76]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)

Intrinsics:
 [[420.04820067   0.         156.17013229]
 [  0.         423.19989828 132.48442139]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[-2.29154134e-01  3.18794487e+00  5.92006807e-03 -4.09074006e-03
  -2.50635145e+01]]
Root mean squared reprojection error:
 0.16919823622862215


### **Pregunta A.1:** Repita el proceso (carga de imágenes, detección y comprobación de esquinas, etc.) para la cámara derecha.

In [77]:
# TODO Homework

# Cargo las imágenes
imgs_right_path = []
for i in range(19):
    if i <10:
        imgs_right_path.append(f'../data/right/right_00{i}.jpg')
    else:
        imgs_right_path.append(f'../data/right/right_0{i}.jpg')

imgs_right = load_images(imgs_right_path)

In [78]:
corners_right_list = []
for i in range(19):
    corners_right = cv2.findChessboardCorners(imgs_right[i],(8,6))
    corners_right_list.append(corners_right)

In [79]:
corners_right_copy = copy.deepcopy(corners_right_list)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)
list_right_imgs_gray = to_gray(imgs_right)
corners_refined_right = refine_corners(list_right_imgs_gray,corners_right_copy,criteria)


In [80]:
imgs_right_copy = copy.deepcopy(imgs_right)

In [81]:
corners_refined_right_copy = corners_refined_right.copy()
imgs_right_copy = imgs_right.copy()
images_right = []
for i in range(19):
    if corners_right_list[i][0]:
        im_right = cv2.drawChessboardCorners(imgs_right_copy[i],(8,6),corners_refined_right_copy[i],corners_right_copy[i][0])
        images_right.append(im_right)
    else:
        print(f"The image {i} doesn't have any valid corners")

The image 14 doesn't have any valid corners


In [82]:
for i in range(18):
        if i <10:
            show_image(f"Right_00{i}",images_right[i])
            write_image(f'../src/right_00{i}.jpg',images_right[i])
        else:
            show_image(f"Right_0{i}",images_right[i])
            write_image(f'../src/right_0{i}.jpg',images_right[i])
         
        

In [83]:
# Filter data and get only those with adequate detections
valid_corners_right = [cor[1] for cor in corners_right_list if cor[0]]
# Convert list to numpy array
valid_corners_right = np.asarray(valid_corners_right, dtype=np.float32)

In [84]:
# TODO You need the points for every image, not just one
chessboard_points_right = get_chessboard_points((8, 6), 30, 30)
valid_chessboard_points_right = [get_chessboard_points((8, 6), 30, 30) for i in range(len(images_right))]

In [85]:
valid_chessboard_points_right = [chessboard_points_right for _ in range(len(valid_corners_right))]


In [86]:
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(valid_chessboard_points_right, valid_corners_right, list_right_imgs_gray[0].shape[::-1], None, None)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

In [87]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)

Intrinsics:
 [[435.42592593   0.         145.73973597]
 [  0.         436.17316265 133.42490959]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[-0.13665117  0.05630508  0.00771191 -0.00736017  0.32078665]]
Root mean squared reprojection error:
 0.19470099997994642


### **Pregunta A.2:** Diferencias entre cv2.findChessboardCorners() y cv2.cornerSubPix()

In [88]:
images_find_corners = []
for i in range(19):
    im = cv2.drawChessboardCorners(imgs[i], (8, 6), corners_list[i][1], corners_list[i][0])
    images_find_corners.append(im)

images_refined_corners = []
for i in range(19):
    im = cv2.drawChessboardCorners(imgs[i], (8, 6), corners_refined[i], corners_list[i][0])
    images_refined_corners.append(im)


show_image("Comparaison",images_refined_corners[18])
show_image("Comparaison_2",images_find_corners[18])
write_image("Comparaison.jpg",images_refined_corners[18])
write_image('Comparaison_2.jpg',images_find_corners[18])

### **Pregunta A.3:** Número mínimo de imágenes necesarias para calibrar.

In [89]:
def calibrate_with_images(images_subset, chessboard_points, corners_list):
    # Lista de puntos válidos para las imágenes seleccionadas
    valid_corners = [corners_list[i][1] for i in range(len(images_subset)) if corners_list[i][0]]
    valid_chessboard_points = [chessboard_points for _ in range(len(valid_corners))]
    
    # Calibración de la cámara con las imágenes seleccionadas
    if len(valid_corners) > 0:
        rms, _, _, _, _ = cv2.calibrateCamera(valid_chessboard_points, valid_corners, images_subset[0].shape[1::-1], None, None)
        return rms
    return None

In [90]:
rms_errors = []
num_images_list = []
for num_images in range(1, 20):
    images_subset = imgs[:num_images]
    rms_error = calibrate_with_images(images_subset, chessboard_points, corners_list)
    
    if rms_error is not None:
        num_images_list.append(num_images)
        rms_errors.append(rms_error)

# Imprimir las listas
print("Números de fotos:", num_images_list)
print("Errores RMS:", rms_errors)

min_rms_error = min(rms_errors)
print(min_rms_error)
for i in range(len(rms_errors)):
    if rms_errors[i] == min_rms_error:
        index = i
num_images = index+1
print(f"Número mínimo de imágenes {num_images}")

Números de fotos: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Errores RMS: [0.36070272974708645, 0.281395401493689, 0.23937924658319015, 0.2235761358303817, 0.213625983864412, 0.2051352107369195, 0.2003704632867212, 0.19576411391589157, 0.19415926825616925, 0.1916311821654824, 0.18785316300405977, 0.1843152903258297, 0.17998969874346216, 0.17624637055336226, 0.1774388472396999, 0.17465275705347968, 0.17214392398257197, 0.17002708090829285, 0.16919823622862215]
0.16919823622862215
Número mínimo de imágenes 19


In [91]:
rms_errors = []
num_images_list = []
for num_images in range(1, 20):
    images_subset = imgs_right[:num_images]
    rms_error = calibrate_with_images(images_subset, chessboard_points_right, corners_right_list)
    
    if rms_error is not None:
        num_images_list.append(num_images)
        rms_errors.append(rms_error)

# Imprimir las listas
print("Números de fotos:", num_images_list)
print("Errores RMS:", rms_errors)

min_rms_error = min(rms_errors)
print(min_rms_error)
for i in range(len(rms_errors)):
    if rms_errors[i] == min_rms_error:
        index = i
num_images = index+1
print(f"Número mínimo de imágenes {num_images}")

Números de fotos: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Errores RMS: [0.1667529544651311, 0.39262784421060903, 0.343450100149161, 0.31238537212417894, 0.28905624732165214, 0.26992717707977953, 0.2577573533773086, 0.24602173529242247, 0.2345453028262974, 0.22603481874686226, 0.21929854050388128, 0.21278420238397905, 0.20701548545957807, 0.2033432518752483, 0.2033432518752483, 0.19903441457091828, 0.19509031446996244, 0.19183187611696378, 0.19470099997994642]
0.1667529544651311
Número mínimo de imágenes 1


## **Apartado B: Corrección de distorsión** (ojo de pez)

En este apartado se trabajará en la corrección de la distorsión debido a lentes de ojo de pez. Primero se calibrará una cámara con este tipo de lente, utilizando las imágenes de la carpeta ``fisheye``. Posteriormente se utilizarán estos parámetros de calibración para corregir la distorsión de una de las imágenes de calibración.

Los pasos que deberá seguir para calibrar una cámara con distorsión de ojo de pez son:

1. Reutilice el método ``load_images()`` para cargar las imágenes de la carpeta ``fisheye``.
2. Detecte las equinas procesando las imágenes con los métodos ``cv2.findChessboardCorners()`` y ``cv2.cornerSubPix()``.
3. Reutilice la función ``get_chessboard_points()`` para obtener las coordenadas del tablero.
4. Defina los argumentos para la función de calibración.
5. Calibre con ``cv2.fisheye.calibrate()``

### **Tarea B.1:** Reutilice el método ``load_images()`` para cargar las imágenes de la carpeta ``fisheye``

In [92]:
# TODO Build a list containing the paths of all images from the fisheye camera and load images
fisheye_imgs_path = []
for i in range(10):
    fisheye_imgs_path.append(f'../data/fisheye/VMRImage{i}.jpg')
fisheye_imgs = load_images(fisheye_imgs_path)


### **Tarea B.2:** Detecte las equinas procesando las imágenes con los métodos ``cv2.findChessboardCorners()`` y ``cv2.cornerSubPix()``.

In [93]:
imgs_corners_fisheye = []
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
for img in fisheye_imgs:
    gray_img_fisheye = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray_img_fisheye, (7, 6), None)
    if ret: 
        refined_corners_fisheye = cv2.cornerSubPix(gray_img_fisheye, corners, (7, 6), (-1, -1), subpix_criteria)
        imgs_corners_fisheye.append(refined_corners_fisheye)

imgs_corners_fisheye = np.array(imgs_corners_fisheye, dtype=np.float32)

### **Tarea B.3:** Reutilice la función ``get_chessboard_points()`` para obtener las coordenadas del tablero

In [94]:
chessboard_dims = (7, 6)
length = 30
fisheye_chessboard_points = [get_chessboard_points(chessboard_dims, length, length) for _ in range(len(imgs_corners_fisheye))]

In [95]:
for i in range(10):
        if i <10:
            show_image(f"fisheye_00{i}",fisheye_imgs[i])
            write_image(f'../src/fisheye_00{i}.jpg',fisheye_imgs[i])
  

### **Tarea B.4:** Defina los argumentos para la calibración

In [96]:
# Parameters for cv2.fisheye.calibrate()
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_CHECK_COND+cv2.fisheye.CALIB_FIX_SKEW
intrinsics = np.zeros((3, 3))
distortion = np.zeros((4, 1))
rotations = [np.zeros((1, 1, 3), dtype=np.float64) for _ in imgs_corners_fisheye]
traslations = [np.zeros((1, 1, 3), dtype=np.float64) for _ in imgs_corners_fisheye]



### **Tarea B.5:** Calibración

In [97]:
rms, _, _, _, _ = \
cv2.fisheye.calibrate(np.expand_dims(np.asarray(fisheye_chessboard_points), -2), imgs_corners_fisheye, gray_img_fisheye.shape[::-1], intrinsics, distortion, rotations, traslations, calibration_flags, subpix_criteria)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\fisheye.cpp:1453: error: (-215:Assertion failed) fabs(norm_u1) > 0 in function 'cv::internal::InitExtrinsics'


In [ ]:
# Show intrinsic matrix and distortion coefficients values
print(intrinsics)
print(distortion)

### **Pregunta B.1:** Corrija la distorsión de las 2 primeras imágenes de la carpeta ``fisheye``

In [ ]:
dim = fisheye_imgs[0].shape[:2][::-1]

map1, map2 = cv2.fisheye.initUndistortRectifyMap(intrinsics, distortion, np.eye(3), intrinsics, dim, cv2.CV_16SC2)

In [ ]:

corrected_imgs = [cv2.remap(img, map1, map2, interpolation=cv2.INTER_CUBIC) for img in fisheye_imgs] 




output_folder = 'corrected_fisheye_img'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)


for idx, img in enumerate(corrected_imgs[:2]):
    output_path = os.path.join(output_folder, f"corrected_fisheye_img_{idx+1}.jpg")
    
    write_image(output_path, img)

    show_image(f"Corrected Image {idx+1}", img)

    cv2.destroyAllWindows()